In [57]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import networkx as nx

In [58]:
def crear_tabla_tipo_cambio(tickers, apikey, secret_key):
    # Esta funcion nos permite crear un DataFrame que contiene como columnas e indices los tickers y en el cuerpo
    # contiene los tipos de cambio entre estas monedas (considerando la conversion, indice -> columna)
    # retorna tabla_df
    global tabla_FX_df
    
    client = Client(apikey, secret_key) #iniciamos el cliente de binance
    
    tipo_cambio = client.get_all_tickers() #obtenemos los datos de binance, estos estan de la forma "{'symbol': 'ETHBTC', 'price': '0.08344700'}"
    tipo_cambio_df = pd.DataFrame(tipo_cambio)
    tipo_cambio_df.set_index('symbol', inplace=True)
    
    table = np.zeros((len(tickers),len(tickers))) # creamos una matriz cuadrada vacia con el tamaño que calce el numero de tickers
    np.fill_diagonal(table, 1) # dado que es una matriz cuadrada de tipo de cambio, la diagonal debe ser de unos (debemos llenarla nosotros porque en los datos no existen pares identicos)
    tabla_df = pd.DataFrame(data = table, index = tickers, columns = tickers)  # creamos la tabla en un data frame

    # El siguiente loop ingresar los datos en el dataframe (tabla_df)
    #  Para esto debemos buscar la combinacion de tickers y asignar a los valores correpsondientes a cada par de monedas

    for i in range(len(tickers)):
        
        for a in range(len(tickers)):
            
            for b in range(len(tipo_cambio_df.index)):
                
                if (tabla_df.columns[i] + tabla_df.index[a]) == tipo_cambio_df.index[b]:
                    tabla_df.loc[tabla_df.index[a], tabla_df.columns[i]] = tipo_cambio_df.iloc[b][0]
                
                elif (tabla_df.index[a] + tabla_df.columns[i]) == tipo_cambio_df.index[b]:
                    tabla_df.loc[tabla_df.columns[i], tabla_df.index[a]] = tipo_cambio_df.iloc[b][0]
        
                else:
                    continue

    # A continuacion vamos a utilizar operaciones matematicas por ende requerimos que los datos sea numericos
    tabla_df = tabla_df._convert(numeric=True)
    # Este segundo loop  nos permite completar la simetria de nuestra tabla de tipo de cambio, puesto que algunos pares pueden estar al reves.
    
    for i in range(len(tickers)): #row
        
        for a in range(len(tickers)): #column
            
            if (tabla_df.iloc[i,a] == 0 and tabla_df.iloc[a,i] != 0):
                tabla_df.iloc[i,a] = 1/tabla_df.iloc[a,i]
            
            else:
                continue
    
    tabla_FX_df = tabla_df


In [68]:
def extraer_edge_list_log(df, edge_list):
    # Esta funcion recibe un data frame de n X n con la info sobre le tipo de cambio y una lista vacia para guardar los valores  los edges y los nodos corrspondientes de la forma:
    # return: ('XRP', 'BTC', 1.652e-05) donde el peso esta en -log()
    for row in range(len(df)): #estamos trabajando con una matriz cuadrada
        for column in range(len(df)):
            if (df.index[row] != df.columns[column] and df.iloc[row, column] != 0): # Tenemos que eliminar las divisiones por 0 ()los pesos que van por ejemplo "ETH-ETH"
                edge_list.append((df.index[row], df.columns[column], -np.log(df.iloc[row, column]))) #creamos un tuple de la forma (tikr1, tikr2, pesos de su conexion que corresponde al tipo de cambio en forma log())
                edge_list.append((df.index[column], df.columns[row], 1/-np.log(df.iloc[row, column]))) #creamos un tuple de la forma (tikr2, tikr1, pesos de su conexion que corresponde al tipo de cambio en forma log())
            else:
                continue

In [61]:
def extraer_edge_list(df, edge_list):
    # Esta funcion recibe un data frame de n X n con la info sobre le tipo de cambio y una lista vacia para guardar los valores  los edges y los nodos corrspondientes de la forma:
    # return: ('XRP', 'BTC', 1.652e-05) donde el peso esta numeric/float64
    for row in range(len(df)): #estamo trabajando con una matriz cuadrada
        for column in range(len(df)):
            edge_list.append((df.index[row], df.columns[column], (df.iloc[row, column]))) #sin el log para poder realizar algunas operaciones
            edge_list.append((df.index[column], df.columns[row], 1/(df.iloc[row, column])))

In [74]:
def search_negative_cycles_log_sum(G, results):
    # Recibe un objeto Graph (G) al cual tiene los pesos de sus edges en formato -log() y una variable AUX para exportar nuestro hallazgos 
    # y calcula los cycles que poseen Oportunidad de arbitrage
    cycle_list = nx.algorithms.cycles.minimum_cycle_basis(G)
    for i in cycle_list:
        if len(i) > 2:
            cycle_sum = 0
            cycle_sum = [G.get_edge_data(i[b], i[b+1])['weight'] if (b < len(i)-1) else G.get_edge_data(i[-1], i[0])['weight'] for b in range(len(i))]
            cycle_sum = np.sum(cycle_sum)
            if cycle_sum < 0:
                results.append((cycle_sum, i))
                
        else:
            continue

In [63]:
def search_positive_cycles_mul(G, Aux):
    # Recibe un objeto Graph (G) al cual tiene los pesos de sus edges en formato numerico y una variable AUX para exportar nuestro hallazgos
    # calcula los cycles que poseen Oportunidad de arbitrage
    cycle_list = nx.algorithms.cycles.minimum_cycle_basis(G)
    for i in cycle_list:
        if len(i) > 2:
            cycle_mul = 0
            cycle_mul = [G.get_edge_data(i[b], i[b+1])['weight'] if (b < len(i)-1) else G.get_edge_data(i[-1], i[0])['weight'] for b in range(len(i))]
            print("chek funcion", cycle_mul, i)
            cycle_mul = np.prod(cycle_mul)
            if cycle_mul > 1:
                Aux.append((cycle_mul, i))
                
        else:
            continue

In [69]:
tabla_FX_df = () # Variable con la cual extraemos los datos de la tavla creada en la funcion crear_tabla_tipo_cambio
tickers = ['XRP', 'BTC', 'ETH', 'USDT', 'USDC', 'ADA', 'BNB', 'LTC', 'NEO', 'SOL']
crear_tabla_tipo_cambio(tickers=tickers, apikey='', secret_key='')

In [70]:
tabla_FX_df.head()

,XRP,BTC,ETH,USDT,USDC,ADA,BNB,LTC,NEO,SOL
XRP,1.000000,61236.987140,5197.505198,1.249063,1.246572,0.000000,685.400960,0.000000,0.000000,0.000000
BTC,0.000016,1.000000,0.084840,0.000020,0.000020,0.000028,0.011189,0.003143,0.000567,0.003945
ETH,0.000192,11.786893,1.000000,0.000240,0.000240,0.000330,0.131800,0.037050,0.006690,0.046550
USDT,0.800600,49022.000000,4159.000000,1.000000,0.999000,1.371000,548.600000,154.000000,27.840000,193.420000
USDC,0.802200,49081.430000,4163.800000,1.001001,1.000000,1.372000,549.300000,153.900000,27.880000,193.580000


In [71]:
edge_list_FX = []
extraer_edge_list_log(tabla_FX_df, edge_list_FX)

In [72]:
edge_list_FX

[('XRP', 'BTC', -11.022506650981901),
 ('BTC', 'XRP', -0.09072346532999538),
 ('XRP', 'ETH', -8.555934019732668),
 ('ETH', 'XRP', -0.11687794666177723),
 ('XRP', 'USDT', -0.2223938324236638),
 ('USDT', 'XRP', -4.49652757498681),
 ('XRP', 'USDC', -0.2203973256461845),
 ('USDC', 'XRP', -4.537260137200362),
 ('XRP', 'BNB', -6.530004009441488),
 ('BNB', 'XRP', -0.1531392627866901),
 ('BTC', 'XRP', 11.0225066509819),
 ('XRP', 'BTC', 0.0907234653299954),
 ('BTC', 'ETH', 2.4669881492856556),
 ('ETH', 'BTC', 0.4053525754834134),
 ('BTC', 'USDT', 10.800024455923422),
 ('USDT', 'BTC', 0.09259238292294202),
 ('BTC', 'USDC', 10.801236034504718),
 ('USDC', 'BTC', 0.09258199680161458),
 ('BTC', 'ADA', 10.484020588726938),
 ('ADA', 'BTC', 0.0953832541186786),
 ('BTC', 'BNB', 4.492824126156553),
 ('BNB', 'BTC', 0.2225771523479294),
 ('BTC', 'LTC', 5.7625775211667065),
 ('LTC', 'BTC', 0.1735334572640226),
 ('BTC', 'NEO', 7.475151254236522),
 ('NEO', 'BTC', 0.13377655728815557),
 ('BTC', 'SOL', 5.535280

In [73]:
##crear un objeto nx.graph
G_FX = nx.Graph()
G_FX.add_weighted_edges_from(edge_list_FX) #añadimos los nodos, los edges y los pesos a la grafica.

In [76]:
results = []
search_negative_cycles_log_sum(G_FX, results)

In [101]:
results

[(-0.19775116349164087, ['ETH', 'BTC', 'XRP']),
 (-0.6254656739909226, ['ETH', 'ADA', 'BTC']),
 (-0.22345181269469105, ['BNB', 'ETH', 'XRP']),
 (-0.8823313268962727, ['LTC', 'ETH', 'BTC']),
 (-0.7388438839932903, ['NEO', 'ETH', 'BTC']),
 (-0.9120391280164939, ['ETH', 'SOL', 'BTC']),
 (-990.4661289126981, ['XRP', 'USDC', 'USDT']),
 (-0.06759310046636141, ['LTC', 'BTC', 'USDT']),
 (-0.08331325545080889, ['SOL', 'BTC', 'USDT']),
 (-0.06755710841210574, ['LTC', 'USDC', 'BTC']),
 (-0.08333269543312033, ['USDC', 'SOL', 'BTC']),
 (-0.1510451648117568, ['ADA', 'BTC', 'BNB']),
 (-1.1836631558371455, ['LTC', 'BTC', 'BNB']),
 (-0.6921616840841942, ['NEO', 'BTC', 'BNB']),
 (-1.362802784471839, ['SOL', 'BTC', 'BNB'])]

In [19]:
# tickers = ['XRP', 'BTC', 'ETH', 'USDT', 'USDC', 'ADA', 'BNB', 'LTC', 'NEO', 'SOL']
# tabla_df = pd.DataFrame()
# edge_list_FX = []
# Arb_op = []

# crear_tabla_tipo_cambio(tabla_df= tabla_df,
# tickers=tickers,
# apikey='',
# secret_key='')

# extraer_edge_list(df= tabla_df, edge_list= edge_list_FX)

# # Creamos la Red
# G_FX = nx.Graph()
# # añadimos los nodos y los pesos
# G_FX.add_weighted_edges_from(edge_list_FX)

# search_positive_cycles_mul(G_FX, Aux=Arb_op)


# print("los edges son:", edge_list_FX)
# print("las oportunidades Son:", Arb_op)

los edges son: []
las oportunidades Son: []


""
